In [1]:
from datasets import load_from_disk, Dataset, concatenate_datasets, ClassLabel, Features, Value, Sequence
from transformers import BertTokenizer, BertForSequenceClassification, BasicTokenizer
from torch.utils.data import  DataLoader
from tqdm.notebook import tqdm
import numpy as np
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [ ]:
augmentation_params = {"n_iter": 1, "p_mask":0.1, "p_pos": 0.3, "p_ng":0.2}

In [ ]:
tokenizer = BasicTokenizer(do_lower_case=True)
DATASET = "dbpedia"

In [5]:
train_data = load_from_disk(f"~/data/{DATASET}/train")
sentences = list(map(lambda e: e["sentence"], train_data))

In [6]:
token_lengths = [len(tokenizer.tokenize(sentence)) for sentence in sentences]


In [7]:
sorted_token_lengths = sorted(token_lengths, reverse=True)
avg_tokens = np.mean(token_lengths)

In [8]:
print(sorted_token_lengths[0:25])
print(avg_tokens)
print(sorted_token_lengths[-25:])

[1498, 1283, 1211, 989, 734, 655, 551, 516, 501, 433, 410, 394, 384, 364, 354, 338, 336, 331, 313, 311, 308, 307, 306, 301, 294]
53.53230357142857
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


In [9]:
pos_tag_word_map_list = base.get_pos_tag_word_map(sentences, tokenizer=tokenizer)

In [10]:
print(train_data[0])

{'label': 8, 'sentence': ' Zarrin Darreh (Persian: زرين دره\u200e also Romanized as Zarrīn Darreh) is a village in Chahriq Rural District Kuhsar District Salmas County West Azerbaijan Province Iran. At the 2006 census its population was 37 in 8 families.'}


In [11]:
augmented_datasets = base.get_augmented_dataset(augmentation_params, train_data, pos_tag_word_map_list, tokenizer=tokenizer, include_idx=False)

In [12]:
print(augmented_datasets[0].keys())

dict_keys(['label', 'sentence'])


In [13]:
aug_datasets_formated = []
ds_schema = Features({
    "sentence": Value("string"),
    "label": ClassLabel(names=["Company", "EducationalInstitution", "Artist", "Athlete", "OfficeHolder", "MeanOfTransportation", "Building", "NaturalPlace", "Village", "Animal", "Plant", "Album", "Film", "WrittenWork"])
})

for iter in augmented_datasets:
    dataset = Dataset.from_dict(iter)
    dataset = dataset.cast(ds_schema)
    aug_datasets_formated.append(dataset)

Casting the dataset:   0%|          | 0/448000 [00:00<?, ? examples/s]

In [14]:
print(aug_datasets_formated[0][70])
print(train_data[70])

{'sentence': "zarrin d ' alba : la : [MASK] is a zarrin as alba - zarrin - romaine ardeche in southeast france . the castle [MASK] [MASK] as a monument historique as 1939 by the french [MASK] of culture .", 'label': 6}
{'label': 6, 'sentence': " Château d'Alba-la-Romaine is a château in Alba-la-Romaine Ardèche in southeast France.The castle is listed as a Monument historique since 1939 by the French Ministry of Culture."}


In [15]:
tokenizer = BertTokenizer.from_pretrained("fabriceyhc/bert-base-uncased-dbpedia_14")
model = BertForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-dbpedia_14", num_labels=14)
model.to(device)
model.eval()

torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/teacher.pth")

In [ ]:
train_dataset = base.prepare_dataset(train_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
train_logits = base.generate_logits(train_dataloader, model)
train_dataset = train_dataset.add_column("logits", train_logits)
train_dataset = train_dataset.remove_columns(["token_type_ids", "attention_mask", "input_ids"])
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

Tokenizing the provided dataset:   0%|          | 0/448000 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/3500 [00:00<?, ?it/s]

In [17]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/448000 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9887633928571429


In [ ]:
aug_clean_datasets = []
for dataset in tqdm(aug_datasets_formated, total=(len(aug_datasets_formated)), desc="Processing augmented datasets: "):
    aug_train_dataset = base.prepare_dataset(dataset, tokenizer)
    aug_train_dataloader = DataLoader(aug_train_dataset, batch_size=128, shuffle=False)
    aug_train_logits = base.generate_logits(aug_train_dataloader, model)
    aug_train_dataset = aug_train_dataset.add_column("logits", aug_train_logits)
    aug_train_dataset = aug_train_dataset.remove_columns(["token_type_ids", "attention_mask", "input_ids"])
    aug_train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

    print(base.check_acc(aug_train_dataset, "Accuracy for augmented dataset: "))

    aug_train_dataset = base.remove_diff_pred_class(train_dataset, aug_train_dataset, pytorch_dataset=False)
    
    print(base.check_acc(aug_train_dataset, "Accuracy for filtered dataset: "))

    aug_train_dataset.reset_format()
    aug_clean_datasets.extend(aug_train_dataset)

Processing augmented datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Tokenizing the provided dataset:   0%|          | 0/448000 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/3500 [00:00<?, ?it/s]

Calculating accuracy based on the saved logits:   0%|          | 0/448000 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.9572946428571428


Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Calculating accuracy based on the saved logits:   0%|          | 0/431575 [00:00<?, ?it/s]

Accuracy for filtered dataset:  0.9915008978740659


In [19]:
print(train_dataset.features)

{'labels': ClassLabel(names=['Company', 'EducationalInstitution', 'Artist', 'Athlete', 'OfficeHolder', 'MeanOfTransportation', 'Building', 'NaturalPlace', 'Village', 'Animal', 'Plant', 'Album', 'Film', 'WrittenWork'], id=None), 'sentence': Value(dtype='string', id=None), 'logits': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}


In [20]:
ds_schema = Features({
    "sentence": Value("string"),
    "labels": ClassLabel(names=["Company", "EducationalInstitution", "Artist", "Athlete", "OfficeHolder", "MeanOfTransportation", "Building", "NaturalPlace", "Village", "Animal", "Plant", "Album", "Film", "WrittenWork"]),
    "logits": Sequence(feature=Value(dtype="float32")),
})

aug_dataset = Dataset.from_list(aug_clean_datasets)
aug_dataset = aug_dataset.cast(ds_schema)


Casting the dataset:   0%|          | 0/431575 [00:00<?, ? examples/s]

In [21]:
aug_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")
train_dataset.set_format(type="torch", columns=["logits", "labels"], device="cpu")

In [22]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))
print(base.check_acc(aug_dataset, "Accuracy for augmented dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/448000 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9887633928571429


Calculating accuracy based on the saved logits:   0%|          | 0/431575 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.9915008978740659


In [23]:
train_all_data = concatenate_datasets([train_dataset, aug_dataset])
train_all_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

In [24]:
print(base.check_acc(train_all_data, "Accuracy for combined dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/879575 [00:00<?, ?it/s]

Accuracy for combined dataset:  0.9901065855668931


In [25]:
print(train_all_data.column_names)

['labels', 'sentence', 'logits']


In [26]:
train_all_data.reset_format()

In [27]:
train_all_data.save_to_disk(f"~/data/{DATASET}/train-logits-augmented")

Saving the dataset (0/1 shards):   0%|          | 0/879575 [00:00<?, ? examples/s]

In [28]:
train_dataset.reset_format()
train_dataset.save_to_disk(f"~/data/{DATASET}/train-logits")

Saving the dataset (0/1 shards):   0%|          | 0/448000 [00:00<?, ? examples/s]

In [29]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval")

eval_dataset = base.prepare_dataset(eval_data, tokenizer)
eval_dataloader = DataLoader(eval_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/112000 [00:00<?, ? examples/s]

In [30]:
test_data = load_from_disk(f"~/data/{DATASET}/test")

test_dataset = base.prepare_dataset(test_data, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/70000 [00:00<?, ? examples/s]

In [ ]:
eval_logits = base.generate_logits(eval_dataloader, model)
test_logits = base.generate_logits(test_dataloader, model)

Generating logits for given dataset:   0%|          | 0/875 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/547 [00:00<?, ?it/s]

In [32]:
eval_dataset.reset_format()
eval_dataset = eval_dataset.add_column("logits", eval_logits)
eval_dataset = eval_dataset.remove_columns(["token_type_ids", "input_ids", "attention_mask"])

In [33]:
test_dataset.reset_format()
test_dataset = test_dataset.add_column("logits", test_logits)
test_dataset = test_dataset.remove_columns(["token_type_ids", "input_ids", "attention_mask"])

In [34]:
eval_dataset.save_to_disk(f"~/data/{DATASET}/eval-logits")
test_dataset.save_to_disk(f"~/data/{DATASET}/test-logits")

Saving the dataset (0/1 shards):   0%|          | 0/112000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/70000 [00:00<?, ? examples/s]

In [35]:
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

eval_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")
test_data.set_format(type="torch", columns=["logits", "labels"], device="cpu")

print(base.check_acc(eval_data, "Accuracy for base eval dataset: "))
print(base.check_acc(test_data, "Accuracy for base test dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/112000 [00:00<?, ?it/s]

Accuracy for base eval dataset:  0.9881785714285715


Calculating accuracy based on the saved logits:   0%|          | 0/70000 [00:00<?, ?it/s]

Accuracy for base test dataset:  0.9883285714285714
